In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import ptb_reader

In [3]:
init_limit = 0.05
max_grad_norm = 5
num_layers = 2
num_steps = 35
hidden_size = 300
max_epoch = 4
max_max_epoch = 13
keep_prob = 0.5
lr_decay = 0.5
batch_size = 20
vocab_size = 10000
learning_rate = 0.001

In [4]:
class TrainModel():
    def __init__(self, reuse = False):
        with tf.variable_scope('model', reuse=reuse):
            self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps], name="input_data")
            self.targets = tf.placeholder(tf.int32, [batch_size, num_steps], name="targets")

            lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)

            cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)

            initializer = tf.random_uniform_initializer(-init_limit, init_limit)

            with tf.device("/cpu:0"):
                embedding = tf.get_variable('embedding', [vocab_size, hidden_size], dtype=tf.float32)
                train_input = tf.nn.embedding_lookup(embedding, self.input_data)

            train_input_drop = tf.nn.dropout(train_input, keep_prob)


            self.initial_state = cell.zero_state(batch_size, tf.float32)

            state =  self.initial_state

            output_arr = []
            state_arr = []
            with tf.variable_scope('RNN', initializer=initializer):
                for step in range(num_steps):
                    if step > 0:
                        tf.get_variable_scope().reuse_variables()

                    (cell_output, state) = cell(train_input_drop[:,step,:], state)

                    output_arr.append(cell_output)
                    state_arr.append(state)

            self.final_state = state_arr[-1]

    #         output = tf.reshape(tf.concat(values=output_arr, axis=1), [-1, hidden_size])
            output = tf.reshape(output_arr[-1], [-1, hidden_size])
            w_out = tf.get_variable("w_out", [hidden_size, vocab_size], dtype=tf.float32)
            b_out = tf.get_variable("b_out", [vocab_size], dtype=tf.float32)

            logits = tf.nn.xw_plus_b(output, w_out, b_out)
    #         targets = tf.reshape(self.targets, [-1])
    #         weights = tf.ones([batch_size * num_steps])
            targets = tf.one_hot(self.targets[:,-1], vocab_size)

            loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    #         loss = -tf.reduce_sum(targets*tf.log(tf.clip_by_value(logits, 1e-10, 1.0)))
    #         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [targets], [weights], vocab_size)

            self.cost = cost = tf.reduce_sum(tf.pow(loss,2))/batch_size

            output_state = self.final_state
            self.train_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


In [5]:
def get_data(filepath):
    word_to_id = ptb_reader._build_vocab(filepath)
    return ptb_reader._file_to_word_ids(filepath, word_to_id), len(word_to_id)

In [6]:
def train_model(data, reuse=False, verbose = True):
    epoch_size = ((len(data) // batch_size) - 1)
    costs = 0.0
    iters = 0
    
    model = TrainModel(reuse = reuse)
    
    saver = tf.train.Saver()
    
    sess = tf.InteractiveSession()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    state = sess.run(model.initial_state)
    
    
    for i in range(2):
        for step, (x, y) in enumerate(ptb_reader.ptb_iterator(data, batch_size, num_steps)):
            
            cost, state, _ = sess.run([model.cost, model.final_state, model.train_optimizer], feed_dict={
                model.input_data: x,
                model.targets: y,
                model.initial_state: state
            })
            costs += cost
            iters += 1

            perplexity = cost

            if step % 100 == 0 and verbose:
                progress = (step / epoch_size) * 100
                print("%d: %.1f%% Perplexity: %.3f (Cost: %.3f) " % (i, progress, perplexity, cost))

    saver.save(sess, './weights/model.ckpt')
    sess.close()
    return (costs / iters), perplexity
    

In [7]:
def test_model(data, verbose = True):
    epoch_size = ((len(data) // batch_size) - 1)
    costs = 0.0
    iters = 0

    saver = tf.train.Saver()
    
    sess = tf.InteractiveSession()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess, './weights/model.ckpt')
    state = sess.run(model.initial_state)
    
    for step, (x, y) in enumerate(ptb_reader.ptb_iterator(data, batch_size, num_steps)):
#         print()
        cost, state, _ = sess.run([model.cost, model.final_state, model.train_optimizer], feed_dict={
            model.input_data: x,
            model.targets: y
#             model.initial_state: state
        })
        peplexity += cost
        iters += 1


        if step % 100 == 0 and verbose:
            progress = (step / epoch_size) * 100
            print("%.1f%% Perplexity: %.3f " % (progress, perplexity))

    sess.close()

    return (perplexity / iters)
    

In [13]:
def main(_):
    global vocab_size
    FLAGS = tf.app.flags.FLAGS
    tf.app.flags.DEFINE_string('test', '../data/ptb.test.txt',"""Path to file for testing """)

    train_data, feat_len = get_data('../data/ptb.train.txt')
    model = None
    model = train_model(train_data, verbose = 1)
    
    test_data, _ = get_data(FLAGS.test)
    perp = test_model(test_data)
    
    print(perp)

In [14]:
if __name__ == "__main__":
    tf.app.run()

ArgumentError: argument --test: conflicting option string: --test

In [ ]:

# train_model(train_data, model)


In [ ]:
# test_model(train_data)

In [ ]:
# tf.reset_default_graph()